In [ ]:
!git clone https://github.com/xinpingwang/paper-with-code.git

In [ ]:
import sys
# 将项目根目录添加到 path 中
sys.path.append('paper-with-code')

导入需要的包

In [ ]:
import os
import numpy as np
import torch
from torch import nn
from torch import optim

import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [ ]:
from lenet.model.lenet import LeNet
from lenet.utils import data_loader, trainer

最大似然损失

In [ ]:
class MLELoss(nn.Module):

    def __init__(self):
        super(MLELoss, self).__init__()

    def forward(self, predict, label):
        return predict.gather(1, label.view(-1, 1)).sum() / predict.size(0)

学习速率

In [ ]:
def get_learning_rate(epoch):
    lr = 0
    if epoch < 3:
        lr = 0.0005
    elif epoch < 6:
        lr = 0.0002
    elif epoch < 9:
        lr = 0.0001
    elif epoch < 13:
        lr = 0.00005
    else:
        lr = 0.00001
    return lr

统计预测正确的数量

In [ ]:
def accuracy_num(predict, label):
    predict = np.argmin(predict.detach().cpu().numpy(), 1)
    label = label.cpu().numpy()
    return int((predict == label).sum())

In [ ]:
MODEL_FILE = 'LeNet.pth'
BATCH_SIZE = 256
EPOCHS = 20

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
train_data_loader, test_data_loader = data_loader.load_data('./data', batch_size=BATCH_SIZE)
model = LeNet()
model.to(device)
criterion = MLELoss()

# LambdaLR 设置的学习速率是：初始学习速率 * lambda 函数返回的结果
optimizer = optim.SGD(model.parameters(), lr=1)
lr_scheduler = optim.lr_scheduler.LambdaLR(optimizer, lr_lambda=lambda epoch: get_learning_rate(epoch + 1))

if os.path.exists(MODEL_FILE):
    model.load_state_dict(torch.load(MODEL_FILE), map_location=device)
    print('loaded model from {}'.format(MODEL_FILE))

训练模型

In [ ]:
train_acc_rates, val_acc_rates = trainer.train(model, train_data_loader, test_data_loader, criterion, optimizer,
                                               accuracy_num, EPOCHS, lr_scheduler=lr_scheduler, checkpoint=MODEL_FILE,
                                               device=device)

可视化训练和验证正确率

In [ ]:
x = np.arange(EPOCHS)
plt.xlabel('epoches')
plt.ylabel('acc rate')
plt.plot(x, train_acc_rates)
plt.plot(x, val_acc_rates)
plt.legend(['training data', 'testing data'], loc='lower right')
plt.show()